***TRANSFER LEARNING USING VGG16 FOR IMAGE CLASSIFICATION***

Dataset - tf_flower_dataset.
used for classify different types of flowers

In [ ]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

In [ ]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load("tf_flowers",split=["train[:70%]", "train[:30%]"],batch_size=-1,as_supervised=True,)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
print(train_labels)

tf.Tensor([2 3 3 ... 0 2 0], shape=(2569,), dtype=int64)


In [ ]:
train_ds[0].shape

TensorShape([442, 1024, 3])

In [ ]:
import tensorflow as tf
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [ ]:
train_ds[0].shape

TensorShape([150, 150, 3])

In [ ]:
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

***load the VGG16 model.***

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

***Add Custom Layers***

In [ ]:
from tensorflow.keras import layers, models
model=models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(50,activation='relu'))
model.add(layers.Dense(20,activation='relu'))
model.add(layers.Dense(5,activation='softmax'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 50)                409650    
                                                                 
 dense_1 (Dense)             (None, 20)                1020      
                                                                 
 dense_2 (Dense)             (None, 5)                 105       
                                                                 
Total params: 15,125,463
Trainable params: 410,775
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 591s 9s/step - loss: 1.9623 - accuracy: 0.4535 - val_loss: 1.1294 - val_accuracy: 0.5623
Epoch 2/50
65/65 [==============================] - 586s 9s/step - loss: 0.8078 - accuracy: 0.6949 - val_loss: 1.0649 - val_accuracy: 0.6284
Epoch 3/50
65/65 [==============================] - 593s 9s/step - loss: 0.5495 - accuracy: 0.8063 - val_loss: 1.0348 - val_accuracy: 0.6984
Epoch 4/50
65/65 [==============================] - 576s 9s/step - loss: 0.3698 - accuracy: 0.8676 - val_loss: 1.1989 - val_accuracy: 0.6498
Epoch 5/50
65/65 [==============================] - 573s 9s/step - loss: 0.2789 - accuracy: 0.9095 - val_loss: 1.1640 - val_accuracy: 0.7179
Epoch 6/50
65/65 [==============================] - 572s 9s/step - loss: 0.1827 - accuracy: 0.9436 - val_loss: 1.1729 - val_accuracy: 0.7179
Epoch 7/50
65/65 [==============================] - 571s 9s/step - loss: 0.1294 - accuracy: 0.9655 - val_loss: 1.2821 - val_accuracy: 0.6907
Epoch 8/50
65

In [ ]:
val_loss,val_acc = model.evaluate(test_ds,test_labels)
print(val_loss,val_acc)

35/35 [==============================] - 249s 7s/step - loss: 0.2077 - accuracy: 0.9319
0.2076687216758728 0.9318801164627075
